In [1]:
import json
import os
import re
from annotationToString import convert_annotations_to_strings

annotations_folder = 'annotations'
annotations_file_1 = 'final_annotations_pedro.json'
annotations_file_2 = 'final_annotations_claudio.json'

file_path_1 = os.path.join(annotations_folder, annotations_file_1)
file_path_2 = os.path.join(annotations_folder, annotations_file_2)

with open(file_path_1, 'r', encoding='utf-8') as f1:
    data_1 = json.load(f1)
f1.close()

with open(file_path_2, 'r', encoding='utf-8') as f2:
    data_2 = json.load(f2)
f2.close()

In [17]:
def analysis_function(data_1, data_2):
    result_for_metric_1 = []
    result_for_metric_2 = []
    disagreement_list = []

    for index in range(len(data_1["annotations"])):
        aux_list = []
        annotation_1 = data_1["annotations"][index]
        annotation_2 = data_2["annotations"][index]

        if (annotation_1 is not None) and (annotation_2 is not None):
            text = annotation_1[0]
            match_length = len(re.match(r'\d_\d+_', text).group())
            text = re.sub(r'\d_\d+_', '', text)

            entities_1 = annotation_1[1]
            entities_2 = annotation_2[1]

            rotulation_1 = convert_annotations_to_strings(text, entities_1, match_length)
            rotulation_2 = convert_annotations_to_strings(text, entities_2, match_length)

            if len(rotulation_1) != 0:
                if(len(rotulation_1) > len(rotulation_2)):
                    bigger = rotulation_1
                    smaller = rotulation_2
                else:
                    bigger = rotulation_2
                    smaller = rotulation_1
                
                agreement = only_one_marked = different_class = different_set = different_set_and_class = 0

                for entity in bigger:
                    disagreement = False
                    if(entity not in smaller):
                        result_for_metric_1.append(entity[0])
                        result_for_metric_2.append(0)
                        if(smaller):
                            for i in range(len(smaller)):
                                if(entity[2] == smaller[i][2]):
                                    different_class += 1     
                                    disagreement = True
                                elif (entity[2][0] == smaller[i][2][0]) and (entity[1] == smaller[i][1]):
                                    different_set += 1
                                    disagreement = True
                                elif (entity[2][0] == smaller[i][2][0]) and (entity[1] != smaller[i][1]):
                                    different_set_and_class +=1
                                    disagreement = True

                                if disagreement:
                                    smaller.remove(smaller[i])
                                    bigger.remove(entity)
                                    break
                                
                        if not disagreement:
                            only_one_marked += 1                          
                    else:
                        agreement +=1
                        smaller.remove(entity)
                        result_for_metric_1.append(entity[0])
                        result_for_metric_2.append(entity[0])
                
                only_one_marked += len(smaller)
                for entity in smaller:
                    result_for_metric_1.append(entity[0])
                    result_for_metric_2.append(0)

                aux_list.extend([agreement, only_one_marked, different_class, different_set, different_set_and_class])
            else:
                aux_list.extend([0, 0, 0, 0, 0])

        disagreement_list.append(aux_list)
    return result_for_metric_1, result_for_metric_2, disagreement_list

In [18]:
from sklearn.metrics import cohen_kappa_score

results = analysis_function(data_1, data_2)

y1 = results[0]
y2 = results[1]
print(y1)
print(y2)

print(cohen_kappa_score(y1, y2))

count0 = 0
count1 = 0


for y in y2:
    if y == 0:
        count0 += 1
    else:
        count1 += 1

print("Discordâncias: " + str(count0))
print("Concordâncias: " + str(count1))

['Fitting room management and occupancy monitoring system', 'customer', 'collecting and tracking data of fitting room activity', 'preset business rules', 'assists retail operations with reduction of theft incidents', 'employee compliance rates', 'notifying staff', 'tracking data', 'X-ray sensing apparatus', 'detector array', 'generate a plurality of signals', 'high-clock logic device', 'detector array', 'low-clock logic device', 'high-clock logic device', 'accelerometer', 'low-clock logic device', 'processor', 'high-clock logic device', 'low-clock logic device', 'receive an acceleration signal', 'determine that the acceleration signal corresponds to a wake gesture', 'determining that the acceleration signal corresponds to the wake gesture', 'send a first power state change signal', 'high-clock logic device', 'a second power state change signal to the high-clock logic device', 'intelligent LED light apparatus', 'intelligent LED light apparatus', 'communication interface', 'processor', '

In [26]:
# Concordância - Só um marcou - Classe Diferente - Conjunto Diferente - Conjunto e Classe Diferente
for result in results[2]:
    print(result[1])

6
11
12
0
7
1
14
2
7
0
5
15
1
6
0
2
0
2
11
8
10
6
4
8
10
7
12
3
3
0
2
7
4
3
0
13
10
0
1
0
3
3
0
5
5
2
1
8
0
0
4
10
12
7
9
0
20
0
10
1
2
5
2
6
6
0
13
8
12
1
0
3
3
3
4
0
2
0
0
3
8
6
7
12
0
6
12
21
0
0
0
0
15
5
9
6
1
0
7
0
6
8
9
2
10
4
0
0
13
9
15
4
6
17
17
1
4
4
7
8
2
1
3
8
0
3
8
0
6
1
0
11
1
0
10
1
1
0
9
0
0
0
0
5
